# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: Ingeniería en Sistemas Computacionales** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 05**: Transformation and actions

**Fecha**: 21/02/2024

**Nombre del Estudiante**:
    - Luis Acosta
    - Samantha Quintero
    - Benjamin Vergara

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQL-Transformations-Actions") \
    .master("spark://be6296989c4d:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/21 14:11:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import importlib
import gatubelxs.spark_utils

importlib.reload(gatubelxs.spark_utils)

columns_info = [ ("track_id", "string"),
                ("track_name", "string"),
                ("artist_id", "string"),
                ("artist_name", "string"),
                ("album_id", "string"),
                ("duration", "integer"),
                ("release_date", "timestamp"),
                ("popularity", "integer"),
                ("danceability", "double"),
                ("energy", "double"),
                ("key", "integer"),
                ("loudness", "double"),
                ("mode", "integer"),
                ("speechiness", "double"),
                ("acousticness", "double"),
                ("instrumentalness", "double"),
                ("liveness", "double"),
                ("valence", "double"),
                ("tempo", "double"),
                ("playlist_id", "string"),
                ("playlist_name", "string"),
                ("duration_mins", "double"),
                ("genre", "string")]

schema = gatubelxs.spark_utils.SparkUtils.generate_schema(columns_info)

# Create DataFrame
tiktok_df = spark \
                .read \
                .schema(schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/tiktok.csv")

Transformations

In [ ]:
grouped_df = tiktok_df.groupBy("genre").avg("duration")

In [ ]:
grouped_df.show()

Actions

In [5]:
row_count=psongs_df.count()
row_count

1023